In [8]:
%load_ext autoreload
%autoreload 2

from ladybugtools_toolkit.box_model.box_model import BoxModel, EnergySettings, EnergySimulation, RecipeSettings, BoxModelFabricProperties, PeopleLoad, LightingLoad, ElectricEquipmentLoad, InfiltrationLoad, SetpointProgram
from honeybee_energy.programtype import ProgramType
from ladybug.sql import SQLiteResult
from honeybee_energy.result.loadbalance import LoadBalance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
epw_path = r"C:\Program Files\IES\Shared Content\Weather\GBR_ENG_London.Wea.Ctr-St.James.Park.037700_TMYx.2004-2018.epw"
folder = r"C:\Users\cbrooker\Desktop\Box_Model\Code_Testing"

box_model = BoxModel(epw = epw_path)

In [4]:
bm_construct_set = BoxModelFabricProperties(epw = epw_path).construction_set
box_model = BoxModel(epw = epw_path, name = "Test",
                        target_sill_height= 0, target_window_height= 2.5, bay_width= 3, count_bays= 10, depth= 10,
                        facade_azimuth_angle= 90,
                        sensor_grid_bay_count= 2,
                        construct_set= bm_construct_set,
                        modifier_set= bm_construct_set.to_radiance_visible_interior())
bm_program_type = ProgramType(identifier='BM_program_type',
                            people = PeopleLoad().load,
                            lighting= LightingLoad().load,
                            electric_equipment= ElectricEquipmentLoad().load,
                            infiltration= InfiltrationLoad(hb_model = box_model.model, ach = 0.1).load,
                            setpoint= SetpointProgram().setpoint,
                            ventilation= None
                            )
box_model.program_type = bm_program_type

simulation = EnergySimulation(model = box_model, epw = box_model.epw, folder = folder,
                                simulation_parameters= EnergySettings().simulation_parameters())

results = simulation.simulate()

In [ ]:
sql_obj.run_periods

In [ ]:
sql_obj.data_collections_by_output_name('Zone Mean Air Temperature')

In [ ]:
# Radiance
recipe_settings = RecipeSettings(folder = folder)